<a href="https://colab.research.google.com/github/maneri13/Compiler-Construction/blob/master/Corona_Dataset_2_Feature_Selection_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Selection Framework

The purpose of this notebook is to make it a bit easier to test feature selection and classification algorithms.

##Imports

In [1]:
import scipy.io 
import numpy as np
import pandas as pd
import csv
import seaborn as sn

from sklearn.model_selection import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import *
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.utils import class_weight
from sklearn.metrics import *
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix

## Data Initialization

If you want to test more models, add them to the models dictionary. **Note: If any model is added to models dictionary, it should have the fit(), predict() and score() methods.** 

If you want to test more feature selection methods, add them to the fs dictionary. **Note: If any FS method is added to the dictionary, it should have fit and transform methods.**

But in both cases, first read the methods in the section below and make sure the required functions are available.

Source: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection

In [ ]:
filename = "Radiomics_Features.csv"

metrics_columns = ['K-fold', 'FS', 'No. of columns', 'Columns selected',
                   'Algorithm', 'Accuracy', 'ROC', 'F1', 'APS',
                   'Sensitivity', 'Specificity', 'Probability']
metrics_df = pd.DataFrame(columns = metrics_columns)

models = [
          ('LogReg', LogisticRegression()), 
          ('RF', RandomForestClassifier()),
          ('KNN', KNeighborsClassifier()),
          ('SVM', SVC(probability = True)), 
          ('GNB', GaussianNB()),
          ('ADA', AdaBoostClassifier()),
          ('XGB', XGBClassifier()),
          # ('Isolation Forest', IsolationForest()),
          # ('Stacking Classifier', StackingClassifier(estimators= 5)),
          # ('Voting Classifier', VotingClassifier(estimators= 5)),
          ('Bagging', BaggingClassifier(base_estimator= RandomForestClassifier())),
          ('Hist Gradient', HistGradientBoostingClassifier())
        ]

fs = [
      ('Chi-Squared(Best 40)', SelectKBest(chi2, k=40)),
      ('F-Classification(Best 40)', SelectKBest(f_classif, k=40)),
      ('FPR test', SelectFpr(chi2, alpha=0.10)),
      ('FDR Test', SelectFdr(f_classif, alpha=0.5)),
      ('FWE Test', SelectFwe(f_classif, alpha=0.5)),
      # ('Variance Threshold', VarianceThreshold(threshold=(.4 * (1 - .4)))),
      ('RFE', RFE(SVR(kernel="linear"), n_features_to_select=5, step=1)),
      # ('Mutual Information Classification', mutual_info_classif(X,y)),
      ('Lasso(SVC)', SelectFromModel(LinearSVC(C=0.1, penalty="l1", dual=False))),
      ('Tree Classifier', SelectFromModel(RandomForestClassifier())),
      ('Logistic Regression', SelectFromModel(LogisticRegression())),
      ('AdaBoost', SelectFromModel(AdaBoostClassifier())),
      ('Extra Trees', SelectFromModel(ExtraTreesClassifier())),
      ('Gradient Boost', SelectFromModel(GradientBoostingClassifier()))
]

In [ ]:
metrics_df.head()

,K-fold,FS,No. of columns,Columns selected,Algorithm,Accuracy,ROC,F1,APS,Sensitivity,Specificity,Probability


## File Upload

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Radiomics_Features.csv to Radiomics_Features.csv


## Initial Dataset Analysis
**Note: Needs to be changed for every data set**

In [ ]:
X = pd.read_csv("Radiomics_Features.csv")
patients = pd.DataFrame(X['Patient ID'], columns=['id'])

patients = X['Patient ID']
y = X['Label_15mm'] 
X = X.drop("Patient ID", axis= 1)
X = X.drop("Label_15mm", axis= 1)


## Store Patient Index

In [ ]:
indices = []
start_value = 0
end_value = -1
count= patients.value_counts().sort_index()
print(count)

for i in range(1,14):
    start_value = end_value + 1
    end_value = start_value + count[i] - 1
    indices.append([start_value, end_value])
print(indices)

1     51
2     28
3     57
4     38
5     38
6     52
7     20
8     16
9     44
10    64
11    67
12    55
13    55
Name: Patient ID, dtype: int64
[[0, 50], [51, 78], [79, 135], [136, 173], [174, 211], [212, 263], [264, 283], [284, 299], [300, 343], [344, 407], [408, 474], [475, 529], [530, 584]]


## Methods

*   data_split: Reads csv and returns X(dataset) and y(target). **Note: Needs to be changed for every data set.**
*   model_test: Takes split data set as 4 inputs. Loops over models dictionary and tests all models. 
*   feature_selection:  Takes split data set as 4 inputs. loops over fs dictionary and tests each feature selection method. Last line of method calls the model_test method in each loop.
*   Main: Splits data into K-folds and runs feature_selection method in loop.



In [ ]:
def data_split():
  X = pd.DataFrame(df_mat, columns=['ch'+str(n) for n in range(0,44)])
  y = pd.DataFrame(mat['classes'], columns= ['class']) 
  X.dropna(inplace=True)
  return X,y

def leave_one_out(df, patient):
    start_value = indices[patient][0]
    end_value = indices[patient][1]
    print("This is the starting index: " + str(start_value) + " of patient " + str(patient))
    
    #Select test and train data by leaving one out
    test = df.iloc[start_value: end_value].values  # select values from start index to end index. 
    idx = df.index.difference(df.index[start_value:end_value])
    train = df.loc[idx].values # Select the values that were not selected in test set

    # Select class values for same indices
    test_class = y.iloc[start_value: end_value].values # select values from start index to end index.
    idx = y.index.difference(y.index[start_value:end_value])
    train_class = y.loc[idx].values #Select the values that were not selected in test set

    # Perform normalization on datasets
    min_max_scaler = preprocessing.MinMaxScaler()
    train = min_max_scaler.fit_transform(train)
    test = min_max_scaler.transform(test)
    
    return test, train, test_class, train_class

def model_test(k_fold, fs, no_columns, sel_columns, X_train, y_train, X_test, y_test):
  for name, model in models:
    clf = model
    print(name)
    clf.fit(X_train, y_train)

    print(y_test)
    predictions = clf.predict(X_test)
    predictions_prob = clf.predict_proba(X_test)
    f1 = f1_score(y_test, predictions) # Error if predict_proba used
    aps = average_precision_score(y_test, predictions_prob[:, 1]) # predict_proba used
    score = clf.score(X_test, y_test)
    try:
      roc = roc_auc_score(y_test, predictions_prob[:, 1]) # predict_proba used
    except:
      roc = 0
    mcm = confusion_matrix(y_test,predictions) # Error if predict_proba used
    tn = mcm[0, 0]
    tp = mcm[1, 1]
    fn = mcm[1, 0]
    fp = mcm[0, 1]
    sensitivity = (tp / (tp + fn))
    specificity = (tn / (tn + fp))

    print('K_fold: ' + str(k_fold))
    print('Accuracy Score: ' + str(score))
    print('Sensitivity: ' + str(sensitivity))
    print('Specificity: ' + str(specificity))
    print('ROC Score: ' + str(roc))
    print('F1 score: ' + str(f1))
    print('Average Precision Score: ' + str(aps))
    row_metrics = pd.DataFrame([
                         [k_fold, fs, no_columns, sel_columns,name,
                          score, roc, f1, aps, sensitivity, specificity,
                          predictions_prob] 
                         ], columns = metrics_columns)
    global metrics_df 
    metrics_df = metrics_df.append(row_metrics, ignore_index=True)


def feature_selection(k_fold, X_train, y_train, X_test, y_test):
  for name, selection in fs:
    print('\nOriginal number of rows and columns in training data set: ' + str(X_train.shape))
    print('Original number of rows and columns in testing data set: ' + str(X_test.shape))
    print('Feature Selection Method: ' + name)
    sel = selection
    sel.fit(X_train, y_train)
    no_col = sum(sel.get_support())
    sel_col = [x for x in X.columns if x  in X.columns[sel.get_support()]]
    print('Number of features selected: ' + str(no_col))
    print( 'Number of features not selected: ' +
     str( len([
        x for x in X.columns
        if x not in X.columns[sel.get_support()]
    ])))
    print('columns selected: ' + str(sel_col))
    X_new_train = sel.transform(X_train)
    X_new_test = sel.transform(X_test)
    print('The new training set has rows and columns: ' + str(X_new_train.shape))
    print('The new testing set has rows and columns: ' + str(X_new_test.shape)+ '\n')
    model_test(k_fold, name, no_col, sel_col, X_new_train, y_train.ravel(), X_new_test, y_test.ravel())

def Main():
  for i in range(0,13):
    print('Leave one out patient = ' + str(i))
    X_test, X_train, y_test, y_train = leave_one_out(X,i)

    print('This is the size of the training set: ' + str(X_train.shape))
    print('This is the size of the testing set: ' + str(X_test.shape))

    feature_selection(i, X_train, y_train, X_test, y_test)
    # count = count + 1

## Program Call

In [ ]:
Main()

Leave one out patient = 0
This is the starting index: 0 of patient 0
This is the size of the training set: (535, 854)
This is the size of the testing set: (50, 854)

Original number of rows and columns in training data set: (535, 854)
Original number of rows and columns in testing data set: (50, 854)
Feature Selection Method: Chi-Squared(Best 40)
Number of features selected: 40
Number of features not selected: 814
columns selected: ['original_shape_MeshVolume', 'original_shape_VoxelVolume', 'original_shape_LeastAxisLength', 'original_shape_MinorAxisLength', 'original_firstorder_Uniformity', 'original_glcm_MaximumProbability', 'original_glrlm_GrayLevelNonUniformityNormalized', 'original_ngtdm_Busyness', 'wavelet-LLH_glrlm_GrayLevelNonUniformity', 'wavelet-LLH_glszm_LowGrayLevelZoneEmphasis', 'wavelet-LLH_glszm_SmallAreaLowGrayLevelEmphasis', 'wavelet-LLH_gldm_SmallDependenceLowGrayLevelEmphasis', 'wavelet-LHL_glrlm_GrayLevelNonUniformity', 'wavelet-LHL_glrlm_LowGrayLevelRunEmphasis', 'w

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 0]
K_fold: 0
Accuracy Score: 0.56
Sensitivity: 0.5714285714285714
Specificity: 0.5
ROC Score: 0.6294642857142857
F1 score: 0.6857142857142857
Average Precision Score: 0.8801919160709161
KNN
[1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 0]
K_fold: 0
Accuracy Score: 0.72
Sensitivity: 0.8095238095238095
Specificity: 0.25
ROC Score: 0.6294642857142857
F1 score: 0.8292682926829269
Average Precision Score: 0.8845939728548424
SVM
[1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 0]
K_fold: 0
Accuracy Score: 0.76
Sensitivity: 0.8095238095238095
Specificity: 0.5
ROC Score: 0.6398809523809523
F1 score: 0.8500000000000001
Average Precision Score: 0.8802574142972935
GNB
[1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 0]
K_fold: 0
Accuracy Score: 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 0]
K_fold: 0
Accuracy Score: 0.72
Sensitivity: 0.7857142857142857
Specificity: 0.375
ROC Score: 0.6428571428571428
F1 score: 0.825
Average Precision Score: 0.897967225929124
RF
[1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 0]
K_fold: 0
Accuracy Score: 0.54
Sensitivity: 0.5476190476190477
Specificity: 0.5
ROC Score: 0.5773809523809523
F1 score: 0.6666666666666667
Average Precision Score: 0.8500710794686414
KNN
[1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 0]
K_fold: 0
Accuracy Score: 0.72
Sensitivity: 0.8095238095238095
Specificity: 0.25
ROC Score: 0.6904761904761905
F1 score: 0.8292682926829269
Average Precision Score: 0.920664075153871
SVM
[1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 0]
K_fold: 0
Accuracy Score: 0.68
Sensitivi

...

[1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
K_fold: 12
Accuracy Score: 0.7777777777777778
Sensitivity: 0.7647058823529411
Specificity: 1.0
ROC Score: 0.9673202614379084
F1 score: 0.8666666666666666
Average Precision Score: 0.9981565275682925
RF
[1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
K_fold: 12
Accuracy Score: 0.7222222222222222
Sensitivity: 0.7058823529411765
Specificity: 1.0
ROC Score: 0.9281045751633987
F1 score: 0.8275862068965517
Average Precision Score: 0.9954767652908608
KNN
[1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
K_fold: 12
Accuracy Score: 0.7592592592592593
Sensitivity: 0.7450980392156863
Specificity: 1.0
ROC Score: 0.803921568627451
F1 score: 0.8539325842696629
Average Precision Score: 0.9846090292604317
SVM
[1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
K_fold: 12
Accuracy Score: 0.7222222222222222
Sensitivity: 0.7058823529411765
Specificity: 1.0
ROC Score: 0.8169934640522876
F1 score: 0.8275862068965517
Average Precision Score: 0.9881822090792192
KNN
[1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
K_fold: 12
Accuracy Score: 0.7037037037037037
Sensitivity: 0.7254901960784313
Specificity: 0.3333333333333333
ROC Score: 0.7483660130718954
F1 score: 0.8222222222222223
Average Precision Score: 0.9811180332407954
SVM
[1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
K_fold: 12
Accuracy Score: 0.7962962962962963
Sensitivity: 0.7843137254901961
Specificity: 1.0
ROC Score: 0.934640522875817
F1 score: 0.8791208791208792
Average Precision Score: 0.9962475487173537
GNB
[1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
K_fold: 12
Accuracy Score: 0.7037037037037037
Sensitivity: 0.6862745098039216
Specificity: 1.0
ROC Score: 0.869281045751634
F1 score: 0.813953488372093
Average Precision Score: 0.9917821607517505
KNN
[1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
K_fold: 12
Accuracy Score: 0.7962962962962963
Sensitivity: 0.8235294117647058
Specificity: 0.3333333333333333
ROC Score: 0.781045751633987
F1 score: 0.8842105263157896
Average Precision Score: 0.9816748802907972
SVM
[1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
K_fold: 12
Accuracy Score: 0.8148148148148148
Sensitivity: 0.803921568627451
Specificity: 1.0
ROC Score: 0.9477124183006536
F1 score: 0.891304347826087
Average Precision Score: 0.9969874324792765
GNB
[1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

## Metrics

In [ ]:
print(metrics_df.shape) #Should be 1170 rows
metrics_df.head(150)


(1404, 12)


,K-fold,FS,No. of columns,Columns selected,Algorithm,Accuracy,ROC,F1,APS,Sensitivity,Specificity,Probability
0,0,Chi-Squared(Best 40),40,"[original_shape_MeshVolume, original_shape_Vox...",LogReg,0.600000,0.613095,0.714286,0.893177,0.595238,0.625000,"[[0.7410681150736975, 0.25893188492630254], [0..."
1,0,Chi-Squared(Best 40),40,"[original_shape_MeshVolume, original_shape_Vox...",RF,0.600000,0.622024,0.705882,0.865814,0.571429,0.750000,"[[0.89, 0.11], [0.32, 0.68], [0.32, 0.68], [0...."
2,0,Chi-Squared(Best 40),40,"[original_shape_MeshVolume, original_shape_Vox...",KNN,0.660000,0.616071,0.773333,0.876405,0.690476,0.500000,"[[1.0, 0.0], [0.0, 1.0], [0.2, 0.8], [0.2, 0.8..."
3,0,Chi-Squared(Best 40),40,"[original_shape_MeshVolume, original_shape_Vox...",SVM,0.600000,0.511905,0.722222,0.810200,0.619048,0.500000,"[[0.7515360824851419, 0.24846391751485816], [0..."
4,0,Chi-Squared(Best 40),40,"[original_shape_MeshVolume, original_shape_Vox...",GNB,0.340000,0.619048,0.377358,0.899413,0.238095,0.875000,"[[0.9999999999878924, 1.2103438784114644e-11],..."
...,...,...,...,...,...,...,...,...,...,...,...,...
145,1,FWE Test,276,"[original_shape_MeshVolume, original_shape_Vox...",RF,0.777778,0.000000,0.000000,NaN,NaN,0.777778,"[[0.73, 0.27], [0.83, 0.17], [0.56, 0.44], [0...."
146,1,FWE Test,276,"[original_shape_MeshVolume, original_shape_Vox...",KNN,0.592593,0.000000,0.000000,NaN,NaN,0.592593,"[[0.4, 0.6], [1.0, 0.0], [0.4, 0.6], [0.4, 0.6..."
147,1,FWE Test,276,"[original_shape_MeshVolume, original_shape_Vox...",SVM,0.851852,0.000000,0.000000,NaN,NaN,0.851852,"[[0.5950202530831717, 0.40497974691682836], [0..."
148,1,FWE Test,276,"[original_shape_MeshVolume, original_shape_Vox...",GNB,0.888889,0.000000,0.000000,NaN,NaN,0.888889,"[[2.1246867646980508e-47, 1.0], [1.0, 1.970502..."


In [ ]:
metrics_df.head()

,K-fold,FS,No. of columns,Columns selected,Algorithm,Accuracy,ROC,F1,APS,Sensitivity,Specificity,Probability
0,0,Chi-Squared(Best 40),40,"[original_shape_MeshVolume, original_shape_Vox...",LogReg,0.60,0.613095,0.714286,0.893177,0.595238,0.625,"[[0.7410681150736975, 0.25893188492630254], [0..."
1,0,Chi-Squared(Best 40),40,"[original_shape_MeshVolume, original_shape_Vox...",RF,0.60,0.622024,0.705882,0.865814,0.571429,0.750,"[[0.89, 0.11], [0.32, 0.68], [0.32, 0.68], [0...."
2,0,Chi-Squared(Best 40),40,"[original_shape_MeshVolume, original_shape_Vox...",KNN,0.66,0.616071,0.773333,0.876405,0.690476,0.500,"[[1.0, 0.0], [0.0, 1.0], [0.2, 0.8], [0.2, 0.8..."
3,0,Chi-Squared(Best 40),40,"[original_shape_MeshVolume, original_shape_Vox...",SVM,0.60,0.511905,0.722222,0.810200,0.619048,0.500,"[[0.7515360824851419, 0.24846391751485816], [0..."
4,0,Chi-Squared(Best 40),40,"[original_shape_MeshVolume, original_shape_Vox...",GNB,0.34,0.619048,0.377358,0.899413,0.238095,0.875,"[[0.9999999999878924, 1.2103438784114644e-11],..."


In [ ]:
fs_columns = ['FS', 'No. of columns',
                   'Accuracy', 'F1', 'APS',
                   'Sensitivity', 'Specificity']
fs_df = pd.DataFrame(columns = fs_columns)



for name, selection in fs:
  a_slice = metrics_df[(metrics_df.FS == name)]
  averages = a_slice.mean()
  row = pd.DataFrame([
                         [name, averages[1], averages[2], averages[3], averages[4],
                          averages[5], averages[6]] 
                         ], columns = fs_columns)
  fs_df = fs_df.append(row, ignore_index=True)
  print(averages)
  # print(name)
  # print(averages[1])

fs_df

K-fold             6.000000
No. of columns    40.000000
Accuracy           0.582084
ROC                0.643492
F1                 0.535535
APS                0.761636
Sensitivity        0.680741
Specificity        0.569492
dtype: float64
K-fold             6.000000
No. of columns    40.000000
Accuracy           0.622061
ROC                0.657803
F1                 0.572453
APS                0.771645
Sensitivity        0.728346
Specificity        0.525558
dtype: float64
K-fold              6.000000
No. of columns    185.000000
Accuracy            0.614807
ROC                 0.658246
F1                  0.538503
APS                 0.744548
Sensitivity         0.672559
Specificity         0.582935
dtype: float64
K-fold              6.000000
No. of columns    702.230769
Accuracy            0.649479
ROC                 0.679924
F1                  0.568568
APS                 0.778431
Sensitivity         0.709666
Specificity         0.604364
dtype: float64
K-fold              6.000000

,FS,No. of columns,Accuracy,F1,APS,Sensitivity,Specificity
0,Chi-Squared(Best 40),40.000000,0.582084,0.643492,0.535535,0.761636,0.680741
1,F-Classification(Best 40),40.000000,0.622061,0.657803,0.572453,0.771645,0.728346
2,FPR test,185.000000,0.614807,0.658246,0.538503,0.744548,0.672559
3,FDR Test,702.230769,0.649479,0.679924,0.568568,0.778431,0.709666
4,FWE Test,300.461538,0.637060,0.670806,0.563233,0.763235,0.704567
5,RFE,5.000000,0.651972,0.681584,0.601097,0.768725,0.769924
6,Lasso(SVC),19.538462,0.645039,0.672548,0.576160,0.763415,0.731972
7,Tree Classifier,290.076923,0.659202,0.683164,0.581703,0.778901,0.726640
8,Logistic Regression,308.000000,0.670768,0.693268,0.601936,0.779709,0.761038
9,AdaBoost,47.153846,0.660260,0.677548,0.591333,0.769441,0.746438


In [ ]:
algo_columns = ['Algorithm', 'No. of columns',
                   'Accuracy', 'F1', 'APS',
                   'Sensitivity', 'Specificity']
algo_df = pd.DataFrame(columns = algo_columns)
print(algo_df.head())
for name, model in models:
  a_slice = metrics_df[(metrics_df.Algorithm == name)]
  averages = a_slice.mean()
  row = pd.DataFrame([
                         [name, averages[1], averages[2], averages[3], averages[4],
                          averages[5], averages[6]] 
                         ], columns = algo_columns)
  algo_df = algo_df.append(row, ignore_index=True)
  # print(averages)
algo_df

Empty DataFrame
Columns: [Algorithm, No. of columns, Accuracy, F1, APS, Sensitivity, Specificity]
Index: []


,Algorithm,No. of columns,Accuracy,F1,APS,Sensitivity,Specificity
0,LogReg,199.365385,0.662391,0.677247,0.601611,0.791974,0.792273
1,RF,199.365385,0.645272,0.675524,0.570674,0.781092,0.700890
2,KNN,199.365385,0.626492,0.648136,0.597433,0.683051,0.793497
3,SVM,199.365385,0.659834,0.683686,0.599312,0.802063,0.769838
4,GNB,199.365385,0.611092,0.681262,0.488647,0.750550,0.566212
5,ADA,199.365385,0.638408,0.664634,0.583825,0.742242,0.737089
6,XGB,199.365385,0.647826,0.679896,0.579831,0.793384,0.734761
7,Bagging,199.365385,0.644075,0.674901,0.575656,0.797370,0.706035
8,Hist Gradient,199.365385,0.655747,0.681996,0.587656,0.784099,0.739581


In [ ]:
patients_columns = ['Patient', 'No. of columns',
                   'Accuracy', 'F1', 'APS',
                   'Sensitivity', 'Specificity']
patients_df = pd.DataFrame(columns = patients_columns)

for i in range(0,13):
  a_slice = metrics_df[(metrics_df['K-fold'] == i)]
  averages = a_slice.mean()
  row = pd.DataFrame([
                         [i, averages[1], averages[2], averages[3], averages[4],
                          averages[5], averages[6]] 
                         ], columns = patients_columns)
  patients_df = patients_df.append(row, ignore_index=True)

patients_df

,Patient,No. of columns,Accuracy,F1,APS,Sensitivity,Specificity
0,0,197.416667,0.614074,0.605944,0.724943,0.881647,0.637346
1,1,197.333333,0.766804,0.000000,0.000000,NaN,NaN
2,2,209.083333,0.555721,0.604334,0.537724,0.707974,0.460718
3,3,193.083333,0.681431,0.624934,0.790827,0.891313,0.766975
4,4,190.666667,0.632883,0.778361,0.564902,0.691508,0.784512
5,5,195.833333,0.344771,0.502290,0.276826,0.226823,0.718107
6,6,198.666667,0.722710,0.806608,0.694172,0.763518,0.797454
7,7,212.750000,0.473457,0.927249,0.209291,0.702929,0.990741
8,8,200.000000,0.758398,0.906854,0.777777,0.896204,0.925000
9,9,194.916667,0.560112,0.551959,0.390553,0.562173,0.321839


In [ ]:
patients_algo_columns = ['Patient','Algorithm', 'No. of columns',
                   'Accuracy', 'F1', 'APS',
                   'Sensitivity', 'Specificity']
patients_algo_df = pd.DataFrame(columns = patients_algo_columns)

for i in range(0,13):
  for name, model in models: 
    a_slice = metrics_df[(metrics_df['K-fold'] == i) & (metrics_df.Algorithm == name)]
    averages = a_slice.mean()
    row = pd.DataFrame([
                         [i,name, averages[1], averages[2], averages[3], averages[4],
                          averages[5], averages[6]] 
                         ], columns = patients_algo_columns)
    patients_algo_df = patients_algo_df.append(row, ignore_index=True)

patients_algo_df

,Patient,Algorithm,No. of columns,Accuracy,F1,APS,Sensitivity,Specificity
0,0,LogReg,197.416667,0.675000,0.622768,0.791377,0.890504,0.742063
1,0,RF,197.416667,0.571667,0.588170,0.694936,0.864349,0.581349
2,0,KNN,197.416667,0.731667,0.660342,0.833089,0.896889,0.801587
3,0,SVM,197.416667,0.690000,0.610367,0.797949,0.866032,0.740079
4,0,GNB,197.416667,0.415000,0.579861,0.491829,0.878464,0.351190
...,...,...,...,...,...,...,...,...
112,12,GNB,200.166667,0.618827,0.916939,0.720611,0.995040,0.598039
113,12,ADA,200.166667,0.740741,0.873638,0.841714,0.991720,0.733660
114,12,XGB,200.166667,0.729938,0.898693,0.833636,0.993735,0.717320
115,12,Bagging,200.166667,0.731481,0.818627,0.833896,0.988746,0.715686


In [ ]:
!pip install xlsxwriter
import xlsxwriter
writer = pd.ExcelWriter('Corona Dataset 2.xlsx', engine='xlsxwriter')
metrics_df.to_excel(writer, sheet_name='Details')
patients_df.to_excel(writer, sheet_name='Patient')
algo_df.to_excel(writer, sheet_name='Algorithm')
fs_df.to_excel(writer, sheet_name= 'Feature Selection' )
patients_algo_df.to_excel(writer, sheet_name= 'Patients_algo')
# from google.colab import drive
# drive.mount('drive')
# metrics_df.to_csv('Corona_Dataset_2.csv')
# !cp Corona_Dataset_2.csv "drive/My Drive/"

In [ ]:
writer.save()
files.download('Corona Dataset 2.xlsx')
# metrics_df.to_csv('Corona_Dataset_2.csv')
# !cp Corona_Dataset_2.csv "drive/My Drive/"

/usr/local/lib/python3.6/dist-packages/xlsxwriter/workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:


def save_average_data(df , file_name):


